### DeGirum PySDK Tutorial for Hailo8
In this notebook, we illustrate the main features of PySDK and how it can be used to quickly develop edge AI applications using the Hailo8 accelerator.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Basic setup
In this notebook, you can set the variable `inference_host_address` to three possible values:
- `@local`: Inference runs on Hailo8 accelerator installed on your machine. 
- `@cloud`: Inference runs on Hailo8 accelerators hosted on DeGirum AI Hub. 
- IP address of AI server: Inference runs on an AI server on a machine with Hailo8 accelerator. If the AI server runs on the same machine as this code, you can specify the IP address as `localhost`

DeGirum hosts a public model zoo `degirum/models_hailort` containing pre-compiled models optimized for Hailo8 devices. If working with an AI server or with a local Hailo8 device, there is no need to register for a DeGirum AI Hub account to access these models. However, if you do not have access to a Hailo8 accelerator but still want to test the models and code, you can use the hardware hosted by DeGirum on our AI Hub. To run inference on DeGirum AI hub, you need to have a token. Please see [this link](https://docs.degirum.com/content/hub/token/) on how to generate an access token. See this [github gist](https://gist.github.com/shashichilappagari/ab856f4ed85fbfb623bc949cf453925b) on how to store and set up the token.

In [ ]:
import degirum as dg, degirum_tools
inference_host_address = "@cloud" # set to @local if you want to run inference on your local machine
zoo_url = "degirum/models_hailort"
token = degirum_tools.get_token() # paste your token here or leave it empty if running on local machine

#### Quick Start
DeGirum's PySDK provides simple APIs to run AI model inference. In general, there are three steps in running an AI model:
1. Loading model
2. Running inference on an input
3. Visualizing inference results

The code block below shows how to get started with PySDK. 

In [ ]:
# import degirum and degirum_tools
import degirum as dg, degirum_tools

# set model name, inference host address, zoo url, token, and image source
model_name = "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1"
image_source='../assets/ThreePersons.jpg'

# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token
)

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

# print('Inference Results \n', inference_result)  # numeric results
print(inference_result)
print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(inference_result)

### Start with local model zoo
In this repo, we provide a `models` folder with a couple of example models. You can use the code block as a reference to run models from a local folder.

In [ ]:
import degirum as dg

hailo_model_zoo = dg.connect(
    inference_host_address='@local',
    zoo_url='../models',    
)

print(hailo_model_zoo.list_models())

In [ ]:
import degirum as dg, degirum_tools

# set model name, inference host address, zoo url, token, and image source
model_name = "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1"
image_source='../assets/ThreePersons.jpg'

# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url='../models',
    token=token
)

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

# print('Inference Results \n', inference_result)  # numeric results
print(inference_result)
print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(inference_result)

#### Numpy Array as `image_source`

In [ ]:
import cv2

image_source='../assets/ThreePersons.jpg'
image_array = cv2.imread(image_source)

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image array")
inference_result = model(image_array)

# print('Inference Results \n', inference_result)  # numeric results
print(inference_result)
print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(inference_result)

#### Image URL as `image_source`

In [ ]:
image_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/ThreePersons.jpg"

# perform AI model inference on given image source
print(f" Running inference using '{model_name}' on image source '{image_source}'")
inference_result = model(image_source)

# print('Inference Results \n', inference_result)  # numeric results
print(inference_result)
print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(inference_result)

#### Unified Output Visualization
- The DeGirum PySDK provides a significant advantage with its **unified output visualization**, simplifying the process of understanding inference results across different model types.

- The `display.show_image(inference_result)` function automatically adapts to the specific AI model type, whether:
  - **Object Detection**: Overlays bounding boxes on detected objects.
  - **Segmentation**: Highlights regions of interest with color-coded masks.
  - **Classification**: Displays the predicted labels or class names.
  - **Keypoints Detection**: Marks keypoints (e.g., facial landmarks or skeletal joints) on the image.

- This unified approach eliminates the need for manual customization, providing a seamless and consistent visualization experience for various AI tasks.

- The code block below demonstrates this advantage using a **widget-based interface**.

In [ ]:
import ipywidgets as widgets
from IPython.display import display as ipy_display, clear_output as ipy_clear_output
import degirum as dg
import degirum_tools

# Define models with categories
model_categories = {
    "Classification": {
        "model_name": "yolov8s_silu_imagenet--224x224_quant_hailort_hailo8_1",
    },
    "Detection": {
        "model_name": "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1",
    },
    "Segmentation": {
        "model_name": "yolov8n_silu_seg--640x640_quant_hailort_hailo8_1",
    },
    "Keypoints": {
        "model_name": "yolov8n_relu6_widerface_kpts--640x640_quant_hailort_hailo8_1",
    },
}

# Widgets for category selection, image path input, and inference
category_dropdown = widgets.Dropdown(
    options=list(model_categories.keys()),
    value="Classification",
    description="Category:",
)

image_path_input = widgets.Text(
    value="../assets/ThreePersons.jpg",
    description="Image Path:",
    placeholder="Enter the path to the image",
)

run_button = widgets.Button(description="Run Inference")
output = widgets.Output()

# Function to handle the button click
def run_inference(change):
    with output:
        ipy_clear_output(wait=True)  # Clear previous outputs
        try:
            # Get selected category and input image path
            category = category_dropdown.value
            model_info = model_categories[category]
            model_name = model_info["model_name"]
            image_source = image_path_input.value
            
            # Display selected category, model, and image path
            print(f"Category: {category}")
            print(f"Model: {model_name}")
            print(f"Image Source: {image_source}")
            
            # Load the model (assuming `inference_host_address`, `zoo_url`, `token` are set)
            print("Loading model...")
            model = dg.load_model(
                model_name=model_name,
                inference_host_address=inference_host_address,
                zoo_url=zoo_url,
                token=token,
            )
            
            # Perform AI model inference
            print("Running inference...")
            inference_result = model(image_source)
            
            # Display results
            print("Inference Results:", inference_result)
            with degirum_tools.Display("AI Camera: Press q to exit") as display:
                display.show_image(inference_result)
        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the function to the button click
run_button.on_click(run_inference)

# Display widgets and output
ipy_display(category_dropdown, image_path_input, run_button, output)


#### Running Inference on Video Stream
- The `predict_stream` function in `degirum_tools` provides a powerful and efficient way to perform AI inference on video streams in real-time. It processes video frames sequentially and returns inference results frame by frame, enabling seamless integration with various video input sources.
- The code below shows how to use `predict_stream` on a video file.

In [5]:
model_name = "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1"
video_source = '../assets/Traffic.mp4'

# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token
)


with degirum_tools.Display("AI Camera") as output_display:
    for inference_result in degirum_tools.predict_stream(model, video_source):
        output_display.show(inference_result)

#### Support for various video sources
- The `predict_stream` function works with different video sources such as webcams, RTSP streams, or video files.

- This code below demonstrates how to run real-time AI inference using `predict_stream`, offering users an intuitive way to interact with DeGirum PySDK.

- Key features of the interface include:
  - **Model Selection**: Users can choose from a list of pre-configured models, such as object detection, segmentation, or keypoints detection.
  - **Video Source Selection**: Users can select from available video sources like webcams, RTSP streams, or video files, with an additional option to specify custom paths or URLs.

- The interface dynamically updates based on user inputs, ensuring seamless configuration of models and video sources.

In [ ]:
# Define available models and video sources
available_models = [
    "yolov8n_relu6_coco--640x640_quant_hailort_hailo8_1",
    "yolov8n_relu6_face--640x640_quant_hailort_hailo8_1",
    "yolov8n_relu6_widerface_kpts--640x640_quant_hailort_hailo8_1",
    "yolov8n_silu_seg--640x640_quant_hailort_hailo8_1",
]

video_sources = {
    "Webcam": "0",  # Default webcam index as string to allow editing
    "RTSP Stream": "rtsp://your_rtsp_stream_address",
    "Sample Video File": "../assets/Traffic.mp4",
}

# Widgets for selecting model and video source
model_dropdown = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model:",
)

video_source_dropdown = widgets.Dropdown(
    options=list(video_sources.keys()),
    value="Webcam",
    description="Source Type:",
)

video_input_text = widgets.Text(
    value=video_sources["Webcam"],
    description="Custom URL/Path:",
    placeholder="Enter webcam index, RTSP Stream, or file path",
)

run_button = widgets.Button(description="Run Inference")
output = widgets.Output()

# Function to handle dropdown change for video source
def on_source_change(change):
    new_value = change["new"] if isinstance(change, dict) else change.new
    if new_value in ["Webcam", "RTSP Stream", "Sample Video File"]:
        video_input_text.disabled = False
        video_input_text.value = video_sources[new_value]
    else:
        video_input_text.disabled = True
        video_input_text.value = str(video_sources[new_value])

# Attach the dropdown change event
video_source_dropdown.observe(on_source_change, names="value")

# Function to handle the button click
def run_inference(change):
    with output:
        ipy_clear_output(wait=True)  # Clear previous outputs
        try:
            # Get selected model and video source
            selected_model = model_dropdown.value
            source_type = video_source_dropdown.value
            video_source = video_input_text.value
            
            # Convert video source to integer if it's the webcam
            if source_type == "Webcam":
                try:
                    video_source = int(video_source)  # Ensure it's an integer
                except ValueError:
                    raise ValueError("Invalid webcam index. It must be an integer.")
            
            # Display selected options
            print(f"Selected Model: {selected_model}")
            print(f"Selected Source Type: {source_type}")
            print(f"Video Source: {video_source}")
            
            # Load the model
            print("Loading model...")
            model = dg.load_model(
                model_name=selected_model,
                inference_host_address=inference_host_address,
                zoo_url=zoo_url,
                token=degirum_tools.get_token(),
            )
            
            # Run AI inference on the video stream
            print("Running inference on video stream. Press 'x' or 'q' to stop.")            
            
            # Display inference results
            with degirum_tools.Display("AI Camera") as display:
                for inference_result in degirum_tools.predict_stream(model, video_source):
                    display.show(inference_result)
        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the function to the button click
run_button.on_click(run_inference)

# Display widgets and output
ipy_display(model_dropdown, video_source_dropdown, video_input_text, run_button, output)

# Initialize the input field based on the default dropdown selection
on_source_change({"new": "Webcam"})  # Passing a dictionary for manual initialization
